In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import requests
from textwrap import shorten
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('PlannedCondosWPB - Sheet1.csv')

In [3]:
df.columns

Index(['Developer', 'Name ', 'Site Address', 'Condo/Rental', 'Units',
       'Stories', 'Price Range', 'Floorplans', 'SF', 'Brokerage',
       'Sales Launch', 'Est. Completion', 'Dev Site Cost', 'Link'],
      dtype='object')

## Geocode

In [4]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [5]:
df['Site Address'] = df['Site Address'] + ' West Palm Beach, FL'

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['Site Address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Link Formatter

In [8]:
df['Story Link'] = 'Click here for story'

df["short_description"] = df["Story Link"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['Story Link'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["Story Link"] = '<a href="' + df["Link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['Story Link']

0     <a href="https://therealdeal.com/miami/2024/03...
1     <a href="https://therealdeal.com/miami/2024/03...
2     <a href="https://therealdeal.com/miami/2023/12...
3     <a href="https://therealdeal.com/miami/2024/03...
4     <a href="https://therealdeal.com/miami/2024/02...
5     <a href="https://therealdeal.com/miami/2023/11...
6     <a href="https://therealdeal.com/miami/2024/02...
7     <a href="https://therealdeal.com/miami/2023/01...
8     <a href="https://therealdeal.com/miami/2024/01...
9     <a href="https://therealdeal.com/miami/2024/02...
10    <a href="https://therealdeal.com/miami/2023/10...
11    <a href="https://therealdeal.com/miami/2023/12...
12    <a href="https://therealdeal.com/miami/2023/12...
13    <a href="https://floridayimby.com/2022/10/one-...
14    <a href="https://therealdeal.com/miami/2023/09...
15    <a href="https://therealdeal.com/miami/2023/08...
16    <a href="https://therealdeal.com/miami/2022/08...
17    <a href="https://therealdeal.com/miami/202

In [16]:
df.columns

Index(['Developer', 'Name ', 'Site Address', 'Condo/Rental', 'Units',
       'Stories', 'Price Range', 'Floorplans', 'SF', 'Brokerage',
       'Sales Launch', 'Est. Completion', 'Dev Site Cost', 'Link', 'geocoded',
       'lat', 'lon', 'Story Link', 'short_description', 'remaining_desc'],
      dtype='object')

In [26]:
def tooltip_html(row_index, max_chars=30):
    """Generate HTML content for the tooltip with line breaks for long text."""
    tooltip_content = '<div class="popup-content">'

    # Dynamically add data for each column
    for column in df.columns:
        value = df.at[row_index, column]
        value_str = str(value).strip()
        if pd.notnull(value) and value_str != "" and column not in ['lat', 'lon', 'Link', 'image_links','short_description','remaining_desc','description_link','geocoded']:

            # Check if the string exceeds the maximum character length
            if len(value_str) > max_chars and column != "Story Link":
                # Insert a <br> tag every max_chars characters
                value_str = '<br>'.join(value_str[i:i+max_chars] for i in range(0, len(value_str), max_chars))

            tooltip_content += f'<strong>{column}:</strong> {value_str}<br>'

    tooltip_content += '</div>'
    return tooltip_content

# Initialize the map
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=12)

# Loop through the DataFrame to create markers
for i in range(len(df)):
    # Create tooltip HTML content
    tooltip_content = tooltip_html(i)
    
    # Create a custom icon
    icon = folium.Icon(color='red')
    
    # Create a marker with a tooltip
    folium.Marker(
        location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
        tooltip=tooltip_content,
        icon=icon  # Specify the custom icon here
    ).add_to(m)

# Display the map
m

In [24]:
# # Assuming df is your DataFrame and is already defined

# def tooltip_html(row_index):
#     """Generate HTML content for the tooltip."""
#     tooltip_content = '<div class="popup-content">'
#     # Add data for each column
#     for column in df.columns:
#         value = df.at[row_index, column]
#         if pd.notnull(value) and str(value).strip() != "" and column not in ['lat', 'lon', 'Link', 'image_links','short_description','remaining_desc','description_link']:
#             # Simplify content for tooltip; you might want to limit the amount of information
#             tooltip_content += f'<strong>{column}</strong>: {value}<br>'
#     tooltip_content += '</div>'
#     return tooltip_content

# # Initialize the map
# m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=12)

# # Loop through the DataFrame to create markers
# for i in range(len(df)):
#     # Create tooltip HTML content
#     tooltip_content = tooltip_html(i)
    
#     # Create a custom icon
#     icon = folium.Icon(color='red')
    
#     # Create a marker with a tooltip
#     folium.Marker(
#         location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
#         tooltip=tooltip_content,
#         icon=icon  # Specify the custom icon here
#     ).add_to(m)

# # Display the map
# m

In [11]:
m.save('index.html')

In [12]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/WPB_Branded_Condos_04_02_24
